In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['repository']


In [2]:
with open("../input/repository/udacity-deep-learning-c6b46a0/seq2seq/data/letters_target.txt") as f:
    target=f.read()

In [3]:
with open("../input/repository/udacity-deep-learning-c6b46a0/seq2seq/data/letters_source.txt") as f:
    data=f.read()

In [4]:
target=target.split("\n")

In [5]:
data=data.split("\n")

In [6]:
len(target)

10000

In [7]:
len(data)

10000

In [8]:
data[:5]

['bsaqq', 'npy', 'lbwuj', 'bqv', 'kial']

In [9]:
target[:5]

['abqqs', 'npy', 'bjluw', 'bqv', 'aikl']

In [10]:
special_words=['<PAD>','<UNK>','<GO>','<EOS>']

In [11]:
set_words=set([charac for sent in data for charac in sent])

In [12]:
words=special_words+list(set_words)

In [13]:
words

['<PAD>',
 '<UNK>',
 '<GO>',
 '<EOS>',
 'a',
 'p',
 's',
 'e',
 'q',
 'n',
 'z',
 't',
 'o',
 'w',
 'c',
 'i',
 'd',
 'x',
 'r',
 'b',
 'm',
 'v',
 'h',
 'g',
 'y',
 'j',
 'k',
 'l',
 'u',
 'f']

In [14]:
vocab_2_int={k:v for v,k in enumerate(words)}

In [15]:
int_2_vocab={v:k for (k,v) in vocab_2_int.items()}

In [16]:
int_2_vocab

{0: '<PAD>',
 1: '<UNK>',
 2: '<GO>',
 3: '<EOS>',
 4: 'a',
 5: 'p',
 6: 's',
 7: 'e',
 8: 'q',
 9: 'n',
 10: 'z',
 11: 't',
 12: 'o',
 13: 'w',
 14: 'c',
 15: 'i',
 16: 'd',
 17: 'x',
 18: 'r',
 19: 'b',
 20: 'm',
 21: 'v',
 22: 'h',
 23: 'g',
 24: 'y',
 25: 'j',
 26: 'k',
 27: 'l',
 28: 'u',
 29: 'f'}

In [17]:
source_letter_ids = [[vocab_2_int.get(letter, vocab_2_int['<UNK>']) for letter in line] for line in data]
 

In [18]:
target_letter_ids=[[vocab_2_int[i] for i in sent]+[vocab_2_int['<EOS>']] for sent in target]

In [19]:
target_letter_ids[:5]

[[4, 19, 8, 8, 6, 3],
 [9, 5, 24, 3],
 [19, 25, 27, 28, 13, 3],
 [19, 8, 21, 3],
 [4, 15, 26, 27, 3]]

In [20]:
source_letter_ids[:5]

[[19, 6, 4, 8, 8],
 [9, 5, 24],
 [27, 19, 13, 28, 25],
 [19, 8, 21],
 [26, 15, 4, 27]]

In [21]:
print(source_letter_ids[0])
print(" ".join([int_2_vocab[i] for i in source_letter_ids[0]]))

[19, 6, 4, 8, 8]
b s a q q


In [22]:
print(target_letter_ids[0])
print(" ".join([int_2_vocab[i] for i in target_letter_ids[0]]))

[4, 19, 8, 8, 6, 3]
a b q q s <EOS>


In [23]:
epochs=100
batchsize=128
rnnsize=60
numlayers=2
encoding_embed=15
decoding_embed=15
lr=0.001

In [24]:
inp=tf.placeholder(tf.int32,[None,None])
target=tf.placeholder(tf.int32,[None,None])
tar_seq_length=tf.placeholder(tf.int32,[None,])
max_target_length=tf.reduce_max(tar_seq_length)
source_seq_length=tf.placeholder(tf.int32,[None,])
lr=tf.placeholder(tf.float32,shape=[])

In [25]:
enc=tf.Variable(tf.random_uniform([len(vocab_2_int),encoding_embed],0,1))
emc_embed=tf.nn.embedding_lookup(enc,inp)


Instructions for updating:
Colocations handled automatically by placer.


In [26]:
def makecell(rnnsize):
    enc_cell=tf.nn.rnn_cell.LSTMCell(rnnsize,dtype=tf.float32)
    return enc_cell

In [27]:
enccell=tf.nn.rnn_cell.MultiRNNCell([makecell(rnnsize) for _ in range(numlayers)])

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


In [28]:
enc_out,enc_last=tf.nn.dynamic_rnn(enccell,emc_embed,sequence_length=source_seq_length,dtype=tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


In [29]:
enc_last

(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 60) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 60) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 60) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_6:0' shape=(?, 60) dtype=float32>))

In [30]:
ending=tf.strided_slice(target,[0,0],[batchsize,-1],[1,1])
decinput=tf.concat([tf.fill([batchsize,1],vocab_2_int['<GO>']),ending],axis=1)

In [31]:
decinput.get_shape()

TensorShape([Dimension(128), Dimension(None)])

In [32]:
target_vocabsize=len(vocab_2_int)

In [33]:
decembeddings=tf.Variable(tf.random_uniform([target_vocabsize,decoding_embed]))
decod_emb_inp=tf.nn.embedding_lookup(decembeddings,decinput)


In [34]:
dec_cell=tf.nn.rnn_cell.MultiRNNCell([makecell(rnnsize) for _ in range(numlayers)])

In [35]:
outlayer=tf.layers.Dense(target_vocabsize,kernel_initializer=tf.truncated_normal_initializer(mean=0.0,stddev=0.1))

In [36]:
with tf.variable_scope("decode"):
    trainhelper=tf.contrib.seq2seq.TrainingHelper(inputs=decod_emb_inp,sequence_length=tar_seq_length,time_major=False)
    traindecoder=tf.contrib.seq2seq.BasicDecoder(dec_cell,trainhelper,initial_state=enc_last,output_layer=outlayer)
    train_decoder_output=tf.contrib.seq2seq.dynamic_decode(traindecoder,impute_finished=True,maximum_iterations=max_target_length)[0]


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [37]:
with tf.variable_scope("decode",reuse=True):
    starttoken=tf.tile(tf.constant([vocab_2_int['<GO>']],dtype=tf.int32),[batchsize])
    inferhelper=tf.contrib.seq2seq.GreedyEmbeddingHelper(decembeddings,starttoken,vocab_2_int['<EOS>'])
    inferdecoder=tf.contrib.seq2seq.BasicDecoder(dec_cell,inferhelper,enc_last,outlayer)
    infer_decoder_output=tf.contrib.seq2seq.dynamic_decode(inferdecoder,impute_finished=True,maximum_iterations=max_target_length)[0]

In [38]:
training_logits=tf.identity(train_decoder_output.rnn_output)
inference_logits=tf.identity(infer_decoder_output.rnn_output)

In [39]:
masks=tf.sequence_mask(tar_seq_length,max_target_length,dtype=tf.float32)

In [40]:
masks

<tf.Tensor 'SequenceMask/Cast_1:0' shape=(?, ?) dtype=float32>

In [41]:
with tf.name_scope("optimization"):
    cost=tf.contrib.seq2seq.sequence_loss(training_logits,target,masks)
    optimizer=tf.train.AdamOptimizer(lr)
    gradients=optimizer.compute_gradients(cost)
    capped_gradients=[(tf.clip_by_value(grad,-5.0,5.0),var) for grad,var in gradients if grad is not None]
    train_op=optimizer.apply_gradients(capped_gradients)

In [42]:
def pad_sequence_batch(sentence_batch,pad_int):
    max_sent=max([len(sent) for sent in sentence_batch])
    return [sentence + [pad_int]*(max_sent-len(sentence)) for sentence in sentence_batch]


In [43]:
def get_batches(targets,sources,batch_size,source_pad_int,target_pad_int):
    for batch_i in range(0,len(sources)//batch_size):
        
        start_i=batch_i*batch_size
        #print(start_i)
        #print(sources)
        #print(targets)
        sources_batch=sources[start_i:(start_i+batch_size)]
        targets_batch=targets[start_i:(start_i+batch_size)]
        #print(len(sources_batch))
        #print(len(targets_batch))
        
        padsources_batch=np.array(pad_sequence_batch(sources_batch,source_pad_int))
        padtarget_batch=np.array(pad_sequence_batch(targets_batch,target_pad_int))
        
        pad_targets_length=[]
        for target in padtarget_batch:
            pad_targets_length.append(len(target))

        pad_source_lengths=[]
        for source in padsources_batch:
            pad_source_lengths.append(len(source))
                
        yield padtarget_batch,padsources_batch,pad_targets_length,pad_source_lengths

In [44]:
trainsource

NameError: name 'trainsource' is not defined

In [45]:
len(source_letter_ids)

10000

In [46]:
trainsource=source_letter_ids[1000:]
traintarget=target_letter_ids[1000:]
validsource=source_letter_ids[:1000]
validtarget=target_letter_ids[:1000]


In [47]:
trainsource[:5]

[[14], [16, 28, 22], [16, 15, 23], [22, 19, 7], [8, 13, 18]]

In [48]:
len(validsource)

1000

In [49]:
max(list(map(len,validsource)))

7

In [50]:
max(list(map(len,validtarget)))

8

In [51]:
len(trainsource)

9000

In [52]:
batchsize

128

In [53]:
learning_rate=0.001

In [54]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
        
    for epoch_i in range(1, epochs+1):
        valid_pad_sid=np.array(pad_sequence_batch(validsource,vocab_2_int['<PAD>']),dtype=np.int32)
        valid_pad_tid=np.array(pad_sequence_batch(validtarget,vocab_2_int['<PAD>']),dtype=np.int32)
        valid_tar_length=np.array([len(i) for i in valid_pad_tid],dtype=np.int32)
        valid_sour_length=np.array([len(i) for i in valid_pad_sid],dtype=np.int32)
        
        for batch_i, (targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(
                get_batches(traintarget, trainsource, batchsize,
                           vocab_2_int['<PAD>'],
                           vocab_2_int['<PAD>'])):
            
            sources_batch=sources_batch.astype(np.int32)
            targets_batch=targets_batch.astype(np.int32)
            
            #print(sources_batch.shape)
            #print(targets_batch.shape)
            # Training step
            sources_lengths=np.array(sources_lengths,dtype=np.int32)
            targets_lengths=np.array(targets_lengths,dtype=np.int32)
            
            #print(sources_lengths.shape)
            #print(sources_lengths[:5])
            _, loss = sess.run(
                [train_op, cost],feed_dict=
                {inp: sources_batch,
                 target: targets_batch,
                 lr: learning_rate,
                 tar_seq_length: targets_lengths,
                 source_seq_length: sources_lengths})
            
            if (batch_i%50)==0:
                print("trainingloss:-",loss ," batch_i: ",batch_i," epoch_i ",epoch_i)
                #print(validsource[:5])
                #print(validtarget[:5])
                
                index=np.random.choice(1000,size=128,replace=False)
                valid_pad_batchsid=valid_pad_sid[index]
                valid_pad_batchtid=valid_pad_tid[index]
                #print(valid_pad_sid[:5])
                #print(valid_pad_tid[:5])
                #print(valid_pad_sid.shape)
                #print(valid_pad_tid.shape)
                
                
                
                valid_tar_length1=np.array([len(i) for i in valid_pad_batchtid],dtype=np.int32)
                valid_sour_length1=np.array([len(i) for i in valid_pad_batchsid],dtype=np.int32)
                #print(np.max(valid_tar_length))
                #print(np.max(valid_sour_length))
                validloss,validpred=sess.run([cost,inference_logits],feed_dict={inp:valid_pad_batchsid,target:valid_pad_batchtid,tar_seq_length:valid_tar_length1,source_seq_length:valid_sour_length1})

                print("validloss:-",validloss)
                
                
                #There is some problem cant use whole validation set as inputs in model. need to figure out why
                # make sure validation set is integer multiple of batchsize for it to work
   
        

                
            

trainingloss:- 3.396941  batch_i:  0  epoch_i  1
validloss:- 3.3655322
trainingloss:- 2.3196027  batch_i:  50  epoch_i  1
validloss:- 2.2663054
trainingloss:- 1.8856539  batch_i:  0  epoch_i  2
validloss:- 1.8706281
trainingloss:- 1.679334  batch_i:  50  epoch_i  2
validloss:- 1.5303653
trainingloss:- 1.4681876  batch_i:  0  epoch_i  3
validloss:- 1.444872
trainingloss:- 1.5244219  batch_i:  50  epoch_i  3
validloss:- 1.31487
trainingloss:- 1.3381524  batch_i:  0  epoch_i  4
validloss:- 1.311339
trainingloss:- 1.3505645  batch_i:  50  epoch_i  4
validloss:- 1.2547642
trainingloss:- 1.1726439  batch_i:  0  epoch_i  5
validloss:- 1.2338535
trainingloss:- 1.177385  batch_i:  50  epoch_i  5
validloss:- 1.0584298
trainingloss:- 1.0233666  batch_i:  0  epoch_i  6
validloss:- 1.0373936
trainingloss:- 1.030014  batch_i:  50  epoch_i  6
validloss:- 0.9934486
trainingloss:- 0.87360895  batch_i:  0  epoch_i  7
validloss:- 0.95297134
trainingloss:- 0.8987556  batch_i:  50  epoch_i  7
validloss:- 0

In [55]:
" ".join([int_2_vocab[i] for i in valid_pad_batchsid[0]])

'a d d u <PAD> <PAD> <PAD>'

In [56]:
validpred1=np.argmax(validpred,axis=-1)

In [57]:
" ".join([int_2_vocab[i] for i in validpred1[0]])

'a d d u <EOS> <PAD> <PAD> <PAD>'

In [58]:
for i in range(5):
    print(" ".join([int_2_vocab[i] for i in valid_pad_batchsid[i]]))

a d d u <PAD> <PAD> <PAD>
y <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
h x r p k j <PAD>
l <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
r j s e t <PAD> <PAD>


In [59]:
for i in range(5):
    print(" ".join([int_2_vocab[i] for i in validpred1[i]]))

a d d u <EOS> <PAD> <PAD> <PAD>
y <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
h j k p r x <EOS> <PAD>
l <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
e j r s t <EOS> <PAD> <PAD>
